In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch

from netam import noof, shmoof
from epam.torch_common import pick_device

In [2]:
all_df = pd.read_csv("/Users/matsen/data/shmoof_edges_11-Jan-2023_NoNode0_iqtree_K80+R_masked.csv")

# Here's the fraction of sequences of length more than 410
(all_df["parent"].str.len() > 410).sum() / len(all_df)

0.00403216242498992

In [3]:
train_df, val_df = shmoof.load_shmoof_dataframes("/Users/matsen/data/shmoof_edges_11-Jan-2023_NoNode0_iqtree_K80+R_masked.csv") #, sample_count=5000)


In [4]:
kmer_length = 5
max_length = 410

train_dataset = shmoof.SHMoofDataset(train_df, kmer_length=kmer_length, max_length=max_length)
val_dataset = shmoof.SHMoofDataset(val_df, kmer_length=kmer_length, max_length=max_length)

print(f"we have {len(train_dataset)} training examples and {len(val_dataset)} validation examples")

we have 35830 training examples and 13186 validation examples


In [5]:
model = noof.NoofModel(train_dataset, embedding_dim=2, nhead=1, dim_feedforward=512, layer_count=3, dropout=0.1)

device = pick_device()
train_dataset.to(device)
val_dataset.to(device)
model.to(device)

burrito = shmoof.NoofBurrito(train_dataset, val_dataset, model, batch_size=1024, learning_rate=0.1, l2_regularization_coeff=1e-6)
print("starting training...")
losses = burrito.train(epochs=100)

Using Metal Performance Shaders


/Users/matsen/mambaforge/envs/epam/lib/python3.9/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


starting training...
Epoch [1/100]	 Loss: 0.062672443	 Val Loss: 0.071376044
Epoch [2/100]	 Loss: 0.062656802	 Val Loss: 0.071376045
Epoch [3/100]	 Loss: 0.062655699	 Val Loss: 0.071376304
Epoch [4/100]	 Loss: 0.062656239	 Val Loss: 0.071376042
Epoch [5/100]	 Loss: 0.062656593	 Val Loss: 0.071375996
Epoch [6/100]	 Loss: 0.062657303	 Val Loss: 0.071376017
Epoch 00006: reducing learning rate of group 0 to 2.0000e-02.
Epoch [7/100]	 Loss: 0.062657595	 Val Loss: 0.071375995
Epoch [8/100]	 Loss: 0.062657396	 Val Loss: 0.071376009
Epoch [9/100]	 Loss: 0.062655896	 Val Loss: 0.071375995
Epoch [10/100]	 Loss: 0.062655913	 Val Loss: 0.071376145
Epoch [11/100]	 Loss: 0.062657752	 Val Loss: 0.071376106
Epoch 00011: reducing learning rate of group 0 to 4.0000e-03.
Epoch [12/100]	 Loss: 0.062658255	 Val Loss: 0.071376185
Epoch [13/100]	 Loss: 0.06265699	 Val Loss: 0.071376001
Epoch [14/100]	 Loss: 0.062657448	 Val Loss: 0.071375996
Epoch [15/100]	 Loss: 0.062656733	 Val Loss: 0.071376007
Epoch [16/

KeyboardInterrupt: 

In [ ]:
torch.save(model, "_ignore/noof_model.pt")